In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import urllib.request
import zipfile
import os
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from datetime import datetime

In [ ]:
!wget --no-check-certificate \
    "https://github.com/ShreyaJaiswal1604/Detecting_Arthritis_X-Ray_Images/archive/refs/heads/main.zip" \
    -O "/tmp/Arthritis-dataset.zip"



zip_ref = zipfile.ZipFile('/tmp/Arthritis-dataset.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

In [ ]:
# Define dataset paths
base_dir = '/tmp/Detecting_Arthritis_X-Ray_Images-main/Image_Datasets/Original_Kaggle_Binary_Dataset'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
val_dir = os.path.join(base_dir, 'val')

In [ ]:
print("base_dir", base_dir)
print("train_dir", train_dir)
print("test_dir", test_dir)
print("val_dir", val_dir)

In [ ]:
# Define image data generators with data augmentation for training and normalization for validation/testing
train_datagen = ImageDataGenerator(
    rescale=1./255,           # Normalize pixel values to [0, 1]
    rotation_range=20,        # Randomly rotate images
    width_shift_range=0.1,    # Randomly shift images horizontally
    height_shift_range=0.1,   # Randomly shift images vertically
    shear_range=0.1,          # Apply shear transformation
    zoom_range=0.1,           # Apply zoom transformation
    horizontal_flip=True,     # Flip images horizontally
    fill_mode='nearest'       # Fill newly created pixels after rotation or shifting
)

In [ ]:
test_val_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale pixel values to [0, 1]


In [ ]:
# Define batch size and image size
batch_size = 32
image_size = (224, 224)

In [ ]:
# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

In [ ]:
test_generator = test_val_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

In [ ]:
val_generator = test_val_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

In [ ]:
# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.applications.DenseNet201(
        include_top=False,
        weights='imagenet',
        input_shape=(224, 224, 3)
    ),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

tb_callback = tf.keras.callbacks.TensorBoard(log_dir="logs/adam", histogram_freq=1)

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model or fit the model

history = model.fit(
    train_generator,
    epochs=3,
    validation_data=val_generator,
    callbacks=[tb_callback]
)


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/fit